In [232]:
import requests

from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import csv

In [261]:
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)

In [66]:
links = []

# URL of the website to scrape
url = 'https://www.accommodationforstudents.com/search-results?location=&beds=0&occupancy=min&propertyTypes=halls&propertyTypes=house&propertyTypes=flat&propertyTypes=studio&minPrice=0&maxPrice=500&geo=false&page=1'

# Send an HTTP GET request to the website
response = requests.get(url)

# Parse the HTML code using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

n = soup.find_all(class_="Link__link--Z9fnL PaginationWithStyles__pageButton--uMeEm")
number_of_pages = int(n[-1].next)

for i in range(1, number_of_pages + 1):
    
    # URL of the website to scrape
    url = f'https://www.accommodationforstudents.com/search-results?location=&beds=0&occupancy=min&propertyTypes=halls&propertyTypes=house&propertyTypes=flat&propertyTypes=studio&minPrice=0&maxPrice=500&geo=false&page={i}'
    
    # Send an HTTP GET request to the website
    response = requests.get(url)
    
    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Access with href
    property = soup.find_all(class_="Link__link--Z9fnL property-card__link")
    hall = soup.find_all(class_="Link__link--Z9fnL student-halls-card__link")
    
    
    for r in property:
        links.append(r['href'])
    for r in hall:
        links.append(r['href'])
    
    time.sleep(1)

In [67]:
print(len(links))

6973


In [263]:
list_of_bills = ['Gas', 'Electricity', 'Water', 'Internet']

list_of_amenities = ['Fridge', 'Freezer', 'Oven', 'Microwave', 'Dishwasher','Washing machine', 'Dryer', 'Furnished',
'Double glazing', 'Separate lounge', 'Bath', 'Shower', 'Alarm', 'Fire alarm', 'Garden', 'Off road parking', 'Garage',
'Wifi', 'Sky/Cable package', 'Energy performance certificate', 'Gas safe registered', 'Electrical safety certificate', 'NRLA registered']

with open('accommodation.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Price", "Bedrooms", "Bathrooms", 'type', 'address', 'deposit', 'available_date', 'rooms_available']
    field = field + list_of_bills + list_of_amenities
    
    writer.writerow(field)

    for l in links:
        link = 'https://www.accommodationforstudents.com' + l
        row = []
    
        # Send an HTTP GET request to the website
        response = requests.get(link)
        # Parse the HTML code using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
    
        if l.split('/')[1].replace('-', ' ') == 'student hall':
            pass
        else:
            # price
            price = soup.find(class_="price-info__amount price-info__amount--rent")
            if price == None:
                row.append(None)
            else:
                row.append(price.contents[2])
    
            # bedrooms
            heading = soup.find(class_="property-header__heading")
            if heading is not None:
                row.append(heading.next.split(',')[0][0])
                # bathrooms
                row.append(heading.next.split(',')[1][1])
                # accomodation type
                row.append(heading.next.split(',')[1].split(' ')[3])
            else:
                row.extend([None]*3)

            # address
            address = soup.find(class_="address__text")
            if address is not None:
                row.append(soup.find(class_="address__text").next)
            else:
                row.append(None)
        
    
            # deposit
            deposit = soup.find(class_="price-info__amount price-info__amount--deposit")
            if deposit is not None:
                row.append(deposit.contents[2])
            else:
                row.append(None)
    
            # available date
            available = soup.find(class_="property-header__availability-date")
            if available is not None:
                row.append(available.contents[2])
            else:
                row.append(None)
    
            # rooms available
            rooms = soup.find(class_="rooms-available__count")
            if rooms is not None:
                row.append(rooms.contents[0])
            else:
                row.append(None)

                
            # bills
            types_of_bills = []
            bills = soup.find(class_="property-terms__bills-label")
            
            if bills is not None:
                bills = bills.next
            else:
                bills = 'Not included'
            
            if bills == 'Bills included':
                types_of_bills = ['Gas', 'Electricity', 'Water', 'Internet']
            elif bills == 'Some bills':
                included = soup.find(class_="PropertyPriceDetailsBillsPackage__services--yiQsz").contents[0]
                types_of_bills = re.sub(r'[^\w]', ' ', included).split(' ')

            for l in list_of_bills:
                if l in types_of_bills:
                    row.append('Yes')
                else:
                    row.append('No')

            # Amenities
            amenities = []
            amenities_list = soup.find_all(class_='feature-list property-details__amenities-list')
            for a in amenities_list:
                types_of_amenities = [li.contents[1] for li in a.find_all(class_="feature-list__item")]
                for amenity in types_of_amenities:
                    if amenity.parent.contents[0].find('path', fill="#6cc2b7") is not None:
                          amenities.append(amenity)
            for l in list_of_amenities:
                if l in amenities:
                    row.append('Yes')
                else:
                    row.append('No')
            writer.writerow(row)


